In [1]:
import os
import sys
import operator
import numpy as np
import pandas as pd
from sklearn import preprocessing
# from sklearn import preprocessing
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.decomposition import PCA
#from Part2_get_data import get_data
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from collections import OrderedDict
from multiprocessing import Process, Lock, Manager
from sklearn.metrics import classification_report

In [2]:
global df1
global df2
#historical_data1_time_Q12006
col_names_svcg = ['loan_sequence_no', 'monthly_reporting_period', 'current_actual_upb', 'current_loan_delinquency_status',
                  'loan_age', 'remaning_months_on_legal_maturity', 'repurchase_flag', 'modification_flag', 'zero_bal_code',
                  'zero_bal_eff_date', 'current_interest_rate', 'current_deferred_upb', 'ddlpi', 'mi_recoveries', 'net_sales_proceeds',
                  'non_mi_recoveries', 'expenses', 'legal_costs', 'maintenance_preservation_cost', 'taxes_insurance', 'misc_expenses',
                  'actual_loss_calc', 'modification_cost']

df1 = pd.read_table('HistoricalInputFiles/historical_data1_time_Q12006.txt',
                    delimiter='|', names=col_names_svcg, index_col=None, nrows=200000, low_memory=False, usecols=list(np.arange(23)))
df2 = pd.read_table('HistoricalInputFiles/historical_data1_time_Q22006.txt',
                    delimiter='|', names=col_names_svcg, index_col=None, nrows=200000, low_memory=False, usecols=list(np.arange(23)))




In [3]:
global X_train
global y_train
global X_test
global y_test
global matrix
global conf_mat_logred
global conf_mat_rf
global conf_mat_nn
global roc_auc_logred
global roc_auc_rf
global roc_auc_nn

In [4]:
def remove_nan(dff):
    dff.current_loan_delinquency_status = dff.current_loan_delinquency_status.replace('R', '1').astype('float64')

    dff.remaning_months_on_legal_maturity = dff.remaning_months_on_legal_maturity.replace(np.nan, 0)
    dff.remaning_months_on_legal_maturity = dff.remaning_months_on_legal_maturity.astype('category')

    dff.repurchase_flag = dff.repurchase_flag.replace(np.nan, 0)
    dff.repurchase_flag = dff.repurchase_flag.astype('category')

    dff.modification_flag = dff.modification_flag.replace(np.nan, 0)
    dff.modification_flag = dff.modification_flag.astype('category')

    dff.zero_bal_code = dff.zero_bal_code.replace(np.nan, 0)
    dff.zero_bal_code = dff.zero_bal_code.astype('category')

    dff.zero_bal_eff_date = dff.zero_bal_eff_date.replace(np.nan, 0)
    dff.zero_bal_eff_date = dff.zero_bal_eff_date.astype('category')

    dff.current_deferred_upb = dff.current_deferred_upb.replace(np.nan, 0)
    dff.current_deferred_upb = dff.current_deferred_upb.astype('category')

    dff.ddlpi = dff.ddlpi.replace(np.nan, 0)
    dff.ddlpi = dff.ddlpi.astype('category')

    dff.mi_recoveries = dff.mi_recoveries.replace(np.nan, 0)

    dff.net_sales_proceeds = dff.net_sales_proceeds.replace(np.nan, 0)
    dff.net_sales_proceeds = dff.net_sales_proceeds.replace('C', 1)
    dff.net_sales_proceeds = dff.net_sales_proceeds.replace('U', 0)
    dff.net_sales_proceeds.astype('float64')

    dff.non_mi_recoveries = dff.non_mi_recoveries.replace(np.nan, 0)

    dff.expenses = dff.expenses.replace(np.nan, 0)

    dff.legal_costs = dff.legal_costs.replace(np.nan, 0)

    dff.maintenance_preservation_cost = dff.maintenance_preservation_cost.replace(np.nan, 0)
    dff.taxes_insurance = dff.taxes_insurance.replace(np.nan, 0)
    dff.misc_expenses = dff.misc_expenses.replace(np.nan, 0)
    dff.actual_loss_calc = dff.actual_loss_calc.replace(np.nan, 0)
    dff.modification_cost = dff.modification_cost.replace(np.nan, 0)

# remove null
remove_nan(df1)
remove_nan(df2)

In [5]:
# #### Processing the data, Change the numerical features to Dummy variables, Create the target variable,  Create Training and Testing datasets
def process_data():
    # Create target variable function
    def f(row):
        if row['current_loan_delinquency_status'] > 0:
            val = 1
        else:
            val = 0
        return val
    # Create dummy variables
    df1_dummies = pd.get_dummies(df1[['repurchase_flag', 'modification_flag']])
    df2_dummies = pd.get_dummies(df2[['repurchase_flag', 'modification_flag']])

    df1_d = df1.drop(['loan_sequence_no', 'repurchase_flag', 'modification_flag'], axis=1)
    df2_d = df2.drop(['loan_sequence_no', 'repurchase_flag', 'modification_flag'], axis=1)

    global df1_final
    global df2_final
    df1_final = pd.concat([df1_d, df1_dummies], axis=1)
    df2_final = pd.concat([df2_d, df2_dummies], axis=1)

    # create target variable
    df1_final['Deliquent'] = df1_final.apply(f, axis=1)
    df2_final['Deliquent'] = df2_final.apply(f, axis=1)

    # Create training and testing set
    train_all = df1_final.drop(['current_loan_delinquency_status'], axis=1)
    test_all = df2_final.drop(['current_loan_delinquency_status'], axis=1)
    
#     X_train = df1_final.drop(['current_loan_delinquency_status', 'Deliquent'], axis=1)
#     y_train = df1_final['Deliquent']

#     X_test = df2_final.drop(['current_loan_delinquency_status', 'Deliquent'], axis=1)
#     y_test = df2_final['Deliquent']

#     X_train = preprocessing.minmax_scale(np.array(X_train).astype(float))  # scale between 0 and 1
#     X_test = preprocessing.minmax_scale(np.array(X_test).astype(float))

    return train_all, test_all

In [6]:
train_all, test_all = process_data()

In [8]:
train_all.to_csv('train_all.csv', index=False)
test_all.to_csv('test_all.csv', index=False)

In [23]:
X = list(X_train.columns)

In [30]:
len(list(X_train.columns))
y = ['Deliquent']

In [9]:
import h2o

h2o.init(strict_version_check=False,port=9957)

Checking whether there is an H2O instance running at http://localhost:9957..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_121"; OpenJDK Runtime Environment (Zulu 8.20.0.5-macosx) (build 1.8.0_121-b15); OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-macosx) (build 25.121-b15, mixed mode)
  Starting server from /anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/hf/c05h_6kd5wlds2ysv1yvpy1c0000gn/T/tmpdo3zi7w5
  JVM stdout: /var/folders/hf/c05h_6kd5wlds2ysv1yvpy1c0000gn/T/tmpdo3zi7w5/h2o_wangying_started_from_python.out
  JVM stderr: /var/folders/hf/c05h_6kd5wlds2ysv1yvpy1c0000gn/T/tmpdo3zi7w5/h2o_wangying_started_from_python.err
  Server is running at http://127.0.0.1:9957
Connecting to H2O server at http://127.0.0.1:9957... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.2
H2O cluster version age:,15 days
H2O cluster name:,H2O_from_python_wangying_kbgw7l
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [10]:
train_df = h2o.import_file('train_all.csv')
test_df = h2o.import_file('test_all.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [20]:
train_df['Deliquent'] = train_df['Deliquent'].asfactor()
test_df['Deliquent'] = test_df['Deliquent'].asfactor()

In [23]:
train_df.describe()

Rows:200000
Cols:25




,monthly_reporting_period,current_actual_upb,loan_age,remaning_months_on_legal_maturity,zero_bal_code,zero_bal_eff_date,current_interest_rate,current_deferred_upb,ddlpi,mi_recoveries,net_sales_proceeds,non_mi_recoveries,expenses,legal_costs,maintenance_preservation_cost,taxes_insurance,misc_expenses,actual_loss_calc,modification_cost,repurchase_flag_0,repurchase_flag_N,repurchase_flag_Y,modification_flag_0,modification_flag_Y,Deliquent
type,int,real,int,int,int,int,real,real,int,int,int,int,int,int,int,int,int,int,real,int,int,int,int,int,enum
mins,200602.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4479.0,-106498.0,-14016.0,-84253.0,-78088.0,-9866.0,-310378.0,0.0,0.0,0.0,0.0,0.0,0.0,
mean,200944.17248499996,148010.13423979998,44.76667500000002,320.3400449999999,0.0212,2805.312715,6.255471470000001,447.3755742,307.660345,12.396625,112.57902,7.914415,-18.064845,-3.69354,-5.64695,-7.853915,-0.87043,-84.159775,14.670465549999998,0.98604,0.01387,9e-05,0.999225,0.000775,
maxs,201803.0,793000.0,145.0,480.0,15.0,201803.0,8.25,172060.01,201801.0,110827.0,519200.0,292124.0,246.0,0.0,0.0,2341.0,426.0,3362.0,131400.3,1.0,1.0,1.0,1.0,1.0,
sigma,296.624968581185,85840.7270786279,35.569494519729645,40.91893713422987,0.2960591127714046,23581.21673081728,0.6023352295978404,6176.556321254537,7859.4999260999375,847.8208627974935,4213.490433332441,1022.6834164769953,767.4886757760426,133.98973262347621,336.7844779894414,423.7957492216717,41.83082177896286,3167.869488546274,856.1131869474124,0.11732513467256762,0.11695166304271797,0.009486429779519258,0.02782810175014162,0.02782810175014162,
zeros,0,2791,3241,138,197209,197209,138,198356,199694,199942,199774,199822,199773,199796,199795,199809,199776,199788,199891,2792,197226,199982,155,199845,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,200603.0,130000.0,0.0,360.0,0.0,0.0,6.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0
1,200604.0,130000.0,1.0,359.0,0.0,0.0,6.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0
2,200605.0,130000.0,2.0,358.0,0.0,0.0,6.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0


In [18]:
X = train_df.col_names[:24]
Y = train_df.col_names[-1]

'Deliquent'

In [25]:
from h2o.automl import H2OAutoML
# Set up AutoML
aml = H2OAutoML()

In [26]:
aml.train(x=X,y=Y,training_frame=train_df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [27]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
aml_leaderboard_df

,model_id,auc,logloss,mean_per_class_error,rmse,mse
0,StackedEnsemble_AllModels_AutoML_20181207_133501,0.911731,0.139201,0.210976,0.187376,0.035110
1,StackedEnsemble_BestOfFamily_AutoML_20181207_1...,0.911731,0.139201,0.210976,0.187376,0.035110
2,DRF_1_AutoML_20181207_133501,0.907181,0.155591,0.219043,0.203407,0.041374
3,XRT_1_AutoML_20181207_133501,0.906353,0.157899,0.215183,0.205562,0.042256
4,GLM_grid_1_AutoML_20181207_133501_model_1,0.745118,0.221634,0.329522,0.241570,0.058356


In [28]:
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [29]:
mod_best

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20181207_133501
No model summary for this model


ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.0275125222633876
RMSE: 0.1658689912653586
LogLoss: 0.10626235967457597
Null degrees of freedom: 180106
Residual degrees of freedom: 180104
Null deviance: 88062.81280740295
Residual deviance: 38277.18962781771
AIC: 38283.18962781771
AUC: 0.958091637980924
pr_auc: 0.6846908221204717
Gini: 0.916183275961848
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.18780698817450475: 


,0,1,Error,Rate
0,165893.0,2238.0,0.0133,(2238.0/168131.0)
1,3483.0,8493.0,0.2908,(3483.0/11976.0)
Total,169376.0,10731.0,0.0318,(5721.0/180107.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.1878070,0.7480513,220.0
max f2,0.1080104,0.7628374,264.0
max f0point5,0.3313988,0.8015241,172.0
max accuracy,0.2191566,0.9687686,207.0
max precision,0.9998833,0.9993280,0.0
max recall,0.0110240,1.0,395.0
max specificity,0.9998833,0.9999941,0.0
max absolute_mcc,0.1950607,0.7324859,217.0
max min_per_class_accuracy,0.0630838,0.8946226,305.0
max mean_per_class_accuracy,0.0703042,0.8980241,297.0


Gains/Lift Table: Avg response rate:  6.65 %, avg score:  6.66 %



...

,0,1,Error,Rate
0,164898.0,3233.0,0.0192,(3233.0/168131.0)
1,4823.0,7153.0,0.4027,(4823.0/11976.0)
Total,169721.0,10386.0,0.0447,(8056.0/180107.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2043321,0.6397460,212.0
max f2,0.1088415,0.6577074,263.0
max f0point5,0.4299877,0.7093129,142.0
max accuracy,0.3385973,0.9578084,167.0
max precision,0.9998772,0.9627093,0.0
max recall,0.0097825,1.0,399.0
max specificity,0.9998772,0.9997086,0.0
max absolute_mcc,0.2249116,0.6187788,204.0
max min_per_class_accuracy,0.0499322,0.8329993,320.0
max mean_per_class_accuracy,0.0649581,0.8425367,301.0


Gains/Lift Table: Avg response rate:  6.65 %, avg score:  6.65 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100052,0.9979190,14.3713367,14.3713367,0.9556049,0.9996119,0.9556049,0.9996119,0.1437876,0.1437876,1337.1336736,1337.1336736
,2,0.0200048,0.9366244,13.1351130,13.7533964,0.8734037,0.9786123,0.9145157,0.9891151,0.1313460,0.2751336,1213.5113045,1275.3396445
,3,0.0300044,0.6721783,11.5318443,13.0130161,0.7667962,0.8181641,0.8652850,0.9321419,0.1153140,0.3904476,1053.1844320,1201.3016101
,4,0.0400040,0.3962600,9.2020945,12.0604179,0.6118823,0.5195895,0.8019431,0.8290181,0.0920174,0.4824649,820.2094454,1106.0417921
,5,0.0500036,0.2588735,7.2230596,11.0930537,0.4802887,0.3171203,0.7376194,0.7266500,0.0722278,0.5546927,622.3059621,1009.3053686
,6,0.1000017,0.0997466,3.3585140,7.2259985,0.2233204,0.1511155,0.4804841,0.4388987,0.1679192,0.7226119,235.8513965,622.5998543
,7,0.1499997,0.0671994,1.3444076,5.2655408,0.0893948,0.0810978,0.3501259,0.3196362,0.0672178,0.7898297,34.4407629,426.5540807
,8,0.2000033,0.0518557,0.7397596,4.1340327,0.0491894,0.0587943,0.2748876,0.2544221,0.0369906,0.8268203,-26.0240436,313.4032677
,9,0.2999994,0.0358625,0.4584347,2.9088787,0.0304831,0.0430093,0.1934224,0.1839538,0.0458417,0.8726620,-54.1565349,190.8878686
,10,0.4000011,0.0253472,0.3498606,2.2691153,0.0232636,0.0304552,0.1508821,0.1455786,0.0349866,0.9076486,-65.0139428,126.9115277


In [30]:
mod_perf = mod_best.model_performance(test_df)
mod_perf


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.07372284008324068
RMSE: 0.2715195022152933
LogLoss: 0.28710582270564294
Null degrees of freedom: 199999
Residual degrees of freedom: 199997
Null deviance: 92813.47637268194
Residual deviance: 114842.32908225719
AIC: 114848.32908225719
AUC: 0.7145940537294438
pr_auc: 0.1388904542988752
Gini: 0.4291881074588877
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.1033622205246096: 


,0,1,Error,Rate
0,160680.0,26963.0,0.1437,(26963.0/187643.0)
1,7798.0,4559.0,0.6311,(7798.0/12357.0)
Total,168478.0,31522.0,0.1738,(34761.0/200000.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.1033622,0.2077987,282.0
max f2,0.0292257,0.3233802,365.0
max f0point5,0.9072939,0.2301102,28.0
max accuracy,0.9998334,0.939925,0.0
max precision,0.9998334,0.6741344,0.0
max recall,0.0102945,1.0,399.0
max specificity,0.9998334,0.9982946,0.0
max absolute_mcc,0.9998334,0.1786554,0.0
max min_per_class_accuracy,0.0433298,0.6436028,343.0
max mean_per_class_accuracy,0.0284777,0.6509346,366.0


Gains/Lift Table: Avg response rate:  6.18 %, avg score:  8.65 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.01,0.9866447,7.3318767,7.3318767,0.453,0.9971714,0.453,0.9971714,0.0733188,0.0733188,633.1876669,633.1876669
,2,0.020005,0.8873566,3.1707057,5.2507711,0.1959020,0.9461028,0.3244189,0.9716307,0.0317229,0.1050417,217.0705656,425.0771146
,3,0.03,0.7247594,2.5666309,4.3565051,0.1585793,0.8059655,0.2691667,0.9164366,0.0256535,0.1306952,156.6630892,335.6505085
,4,0.040005,0.5712060,1.9169828,3.7463958,0.1184408,0.6453605,0.2314711,0.8486422,0.0191794,0.1498746,91.6982756,274.6395826
,5,0.05,0.4318190,1.9027074,3.3778425,0.1175588,0.4997319,0.2087,0.7788950,0.0190176,0.1688921,90.2707443,237.7842518
,6,0.1,0.1719443,1.8985191,2.6381808,0.1173,0.2663726,0.163,0.5226338,0.0949260,0.2638181,89.8519058,163.8180788
,7,0.15001,0.1084825,1.8301754,2.3688097,0.1130774,0.1355429,0.1463569,0.3935863,0.0915271,0.3553451,83.0175358,136.8809736
,8,0.2,0.0812213,1.4440049,2.1376548,0.0892178,0.0930732,0.132075,0.3184731,0.0721858,0.4275310,44.4004913,113.7654771
,9,0.3,0.0541062,1.3004775,1.8585957,0.08035,0.0658557,0.1148333,0.2342673,0.1300477,0.5575787,30.0477462,85.8595668
,10,0.40043,0.0406178,1.0918495,1.6662916,0.0674599,0.0463507,0.1029518,0.1871368,0.1096544,0.6672331,9.1849516,66.6291605
